In [1]:
!pip install autollm gradio gitpython nbconvert -Uqq

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 25.3/25.3 MB 37.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.6/190.6 kB 15.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 256.7/256.7 kB 22.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 795.5/795.5 kB 41.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 51.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.5/59.5 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.9/92.9 kB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.7/75.7 kB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.4/60.4 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 53.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.0/77.0 kB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 75.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
from autollm import AutoQueryEngine
from autollm.utils.document_reading import read_github_repo_as_documents, read_files_as_documents
import os
import gradio as gr

In [3]:
os.environ["OPENAI_API_KEY"] = ""

## Read files as documents

In [5]:
git_repo_url = "https://github.com/langchain-ai/langchain.git"
required_exts = [".md"]

In [6]:
documents = read_github_repo_as_documents(git_repo_url=git_repo_url, relative_folder_path="docs", required_exts=required_exts)

Temporary directory created at autollm/temp


## AutoQueryEngine

In [7]:
query_engine = AutoQueryEngine.from_parameters(documents=documents)

[nltk_data] Downloading package punkt to /tmp/llama_index...
[nltk_data]   Unzipping tokenizers/punkt.zip.


Parsing documents into nodes:   0%|          | 0/25 [00:00<?, ?it/s]

Generating embeddings:   0%|          | 0/49 [00:00<?, ?it/s]

Embedding Token Usage: 8493
Embedding Token Usage: 9426
Embedding Token Usage: 6262
Embedding Token Usage: 5021
Embedding Token Usage: 3440


In [8]:
system_prompt = "You are an AI assitant that helps users find the most relevant and accurate answers to their questions."

query_wrapper_prompt = """
The document information is below.
-------------------------------
{context_str}
-------------------------------
Using the document information and mostly relying on it,
answer the query.
Query: {query_str}
Answer:
"""

In [9]:
enable_cost_calculator = True

In [10]:
model = "gpt-3.5-turbo"

In [11]:
vector_store_type = "LanceDBVectorStore"

In [12]:
uri= "tmp/lancedb"
table_name = "vectors"

In [13]:
chunk_size = 1024

In [14]:
similarity_top_k = 5

In [15]:
llm_params = {"model": model}

In [16]:
vector_store_params = {"vector_store_type": vector_store_type, "uri":uri, "table_name":table_name}

In [17]:
service_context_params = {"chunk_size": chunk_size}

In [18]:
query_engine_params = {"similarity_top_k": similarity_top_k}

In [21]:
query_engine_new = AutoQueryEngine.from_parameters(documents=documents, system_prompt=system_prompt, query_wrapper_prompt=query_wrapper_prompt, enable_cost_calculator=enable_cost_calculator, llm_params=llm_params, vector_store_params = vector_store_params, service_context_params=service_context_params, query_engine_params=query_engine_params)

Parsing documents into nodes:   0%|          | 0/25 [00:00<?, ?it/s]

Generating embeddings:   0%|          | 0/49 [00:00<?, ?it/s]

Embedding Token Usage: 8493
Embedding Token Usage: 9426
Embedding Token Usage: 6262
Embedding Token Usage: 5021
Embedding Token Usage: 3440


## Ask your Question

In [23]:
response = query_engine_new.query("What do we mean by Chain in Langchain?")

Embedding Token Usage: 10
LLM Prompt Token Usage: 2485
LLM Completion Token Usage: 80
LLM Total Token Cost: $0.003887


In [24]:
response

Response(response='In the context of LangChain, a "Chain" refers to a sequence of components that are executed in order to perform a specific task or achieve a desired outcome. These components can include data processing steps, machine learning models, or any other functionality that is needed to complete the task. The Chain in LangChain allows users to define and customize the sequence of components to suit their specific needs.', source_nodes=[NodeWithScore(node=TextNode(id_='bbd4538e-34b5-414b-a8e0-fba7d87baf44', embedding=None, metadata={}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='autollm/temp/docs/docs/community.md_part_0', node_type=None, metadata={}, hash=None)}, hash='66335aa3890a0c8b612ca51b121244a631ab3d1cdefef16739479244c6f6b774', text='# Community navigator\n\nHi! Thanks for being here. We’re lucky to have a community of so many passionate developers building with LangChain–we ha

In [25]:
print(response.response)

In the context of LangChain, a "Chain" refers to a sequence of components that are executed in order to perform a specific task or achieve a desired outcome. These components can include data processing steps, machine learning models, or any other functionality that is needed to complete the task. The Chain in LangChain allows users to define and customize the sequence of components to suit their specific needs.


In [26]:
import gradio as gr
def chat_function(query):
  return query_engine.query(query).response

In [27]:
demo = gr.Interface(fn=chat_function, inputs="text", outputs = "text")
demo.launch(share=True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://1edaabc88a4d7151b7.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
